In [ ]:
!pip install kaggle
!pip install rouge-score jiwer
!pip install nltk 
!pip install pandas 
!pip install numpy 
!pip install spacy 
!pip install sklearn 
!pip install matplotlib 
!pip install pillow
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import shutil

print("Imported Successfully")

Importing Images datasets from Kaggle

In [ ]:
!kaggle datasets download -d eeshawn/flickr30k

# Check if the 'dataset' folder exists. If not, create it.
if not os.path.exists('dataset'):
    os.makedirs('dataset')

# Unzip the file into the 'dataset' folder.
# !unzip flickr30k.zip -d dataset

Importing text data from Github

In [ ]:
# !wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip -P dataset/
# !unzip dataset/Flickr8k_text.zip -d dataset/

Delete zip files and keep unzipped folders and files only

In [ ]:
# shutil.rmtree('dataset/__MACOSX', ignore_errors=True)
# if os.path.exists('dataset/Flickr8k_Dataset.zip'):
#     os.remove('dataset/Flickr8k_Dataset.zip')
# if os.path.exists('dataset/Flickr8k_text.zip'):
#     os.remove('dataset/Flickr8k_text.zip')

In [ ]:
!ls /workspace

In [ ]:
image_data_location = "/workspace/flickr30k_images"
caption_data_location = "/workspace/captions.txt"

print(os.path.join(image_data_location))

It seems some images are not good , Collecting all the images which are opening.

In [ ]:
#  to collect images from image_data_location folder which are getting opened

images_good = []
# iterate over all the files in the image_data_location
for filename in os.listdir(image_data_location):
  # check if the file is an image
  if filename.endswith(('.jpg', '.jpeg', '.png')):
    # open the image
    try:
      img = Image.open(os.path.join(image_data_location, filename))
      # images_good.append(filename.split(".")[0]) #append to a list
      images_good.append(filename) #append to a list
    except IOError:
      print("Unable to open image:", filename)

# Now you have a list 'images' containing opened image objects from the specified directory.
# You can further process these images as needed.
print(f"Loaded {len(images_good)} images")

Collecting the dataframe from raw data

In [ ]:
raw_df = pd.read_csv(caption_data_location, sep=",", header=None, names=['image','comment_number','captions'], skiprows=1)
raw_df.head(10)

In [ ]:
# The below code is to clear out the part post the image name ex: 1000268201_693b08cb0e.jpg#0	 --> 1000268201_693b08cb0e.jpg
# raw_df['image'] = raw_df['image'].str.split('.').str[0]

Compare good images list with dataframe and identifying the indexes of bad images

In [ ]:
#  to find the df[image] rows not in images_good list
# apprently 30K data set is good. So no need to do the below code.

# indices_not_in_list = []
# for index, image_name in raw_df['image'].items():
#     if image_name not in images_good:
#         # print(image_name)
#         indices_not_in_list.append(index)

In [ ]:
# print(indices_not_in_list)
# len(indices_not_in_list)

Dropping bad images captions from captions dataframe

In [ ]:
# df = raw_df.drop(indices_not_in_list, inplace=False)
df = raw_df.drop('comment_number', axis=1)

In [ ]:
print(df.head())
print(f"raw df shape - {raw_df.shape}")
print(f"new df shape - {df.shape}")

Find out the caption which is having maximum words

In [ ]:
# to find the maximum words string from df['captions'] column

df['word_count'] = df['captions'].apply(lambda x: len(x.split()))
max_words_string = df.loc[df['word_count'].idxmax(), 'captions']
print(f"The string with the maximum words is:\n{max_words_string}")
print(f"the length of the string is {len(max_words_string.split())}")


randomly printing an image and caption

In [ ]:
data_idx = 11
image_name = df.iloc[data_idx,0]
# print(image_name)
image_path = image_data_location + "/" + image_name
print(image_path)
img = mpimg.imread(image_path)
plt.imshow(img)
plt.show()

for i in range(data_idx, data_idx+5):
    print(f"Caption - {df.iloc[i,1]}")

Spacy library :
spaCy is used for Tokenization: Breaks down text into individual words or tokens.








In [ ]:
from collections import Counter
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T

In [ ]:
!python -m spacy download en_core_web_sm


import spacy

try:
    spacy_eng = spacy.load('en_core_web_sm')
    print("Model loaded successfully!")
except OSError:
    print("Failed to load the model. Try reinstalling.")
    

In [ ]:
spacy_eng = spacy.load('en_core_web_sm')
text = "This is a good place to find a city"
[token.text.lower() for token in spacy_eng.tokenizer(text)]

Class to add special tokens , tokenization and preprocessing

In [ ]:
class Vocabulary:
    def __init__(self,freq_threshold):
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
        self.stoi = {v:k for k,v in self.itos.items()}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_eng.tokenizer(text)]

    def build_vocab(self,sentence_list):
        frequencies = Counter()
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenize(sentence):
                frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self,text):
        tokenized_text = self.tokenize(text)
        return [self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text]



In [ ]:
v = Vocabulary(freq_threshold=1) #was1
v.build_vocab(["This is a new city"])
print(v.stoi)
print(v.itos)
print(v.numericalize("This is a new city"))

Dataset preparation
Images and Captions dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,root_dir,captions_df,transform=None, freq_threshold=5): # was freq_threshold=5
        self.root_dir = root_dir
        self.df = captions_df


        self.transform = transform
        self.imgs = self.df["image"] .astype(str)
        self.captions = self.df["captions"]

        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocab(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        caption = self.captions[idx]
        img_name = self.imgs[idx]

        img_location = os.path.join(self.root_dir,img_name)
        img = Image.open(img_location).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        caption_vec = []
        caption_vec += [self.vocab.stoi["<SOS>"]]
        caption_vec += self.vocab.numericalize(caption)
        caption_vec += [self.vocab.stoi["<EOS>"]]
        #print(f"caption - {caption}")

        return img, torch.tensor(caption_vec)

In [ ]:
#defing the transform to be applied
transforms = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [ ]:
def show_image(inp, title=None):
    """Imshow for Tensor"""
    inp = inp.numpy().transpose((1,2,0))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
# testing the dataset
dataset = CustomDataset(
        root_dir = image_data_location,
        captions_df = df,
        transform = transforms
)

In [ ]:
def token_to_sentence(dataset, caps):
  special_strings = ["<SOS>", "<UNK>", "<EOS>","<PAD>"]
  sen = [dataset.vocab.itos[token] for token in caps.tolist()]
  sen = [token for token in sen if token not in special_strings]
  return " ".join(sen)

In [ ]:
img, caps = dataset[0]
show_image(img,"Image")
print("Token :", caps)
print("Sentence: ")
sen = token_to_sentence(dataset, caps)
print(sen)
# sen = [dataset.vocab.itos[token] for token in caps.tolist()]
# special_strings = ["<SOS>", "<UNK>", "<EOS>"]
# sen = [token for token in sen if token not in special_strings]
# print(" ".join(sen))

Collate function in a PyTorch

1) Collate Images: Combine a batch of images into a single tensor.

2) Pad Captions: Pad captions to a uniform length using a special padding token.

In [ ]:
class CapsCollate:
    def __init__(self,pad_idx,batch_first=False):
        self.pad_idx = pad_idx
        self.batch_first = batch_first

    def __call__(self,batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
#         print(f"shape - {(imgs)}")
#         print("----"*22)
        imgs = torch.cat(imgs,dim=0)
#         print(f"shape - {imgs}")
#         print("------")
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=self.batch_first, padding_value=self.pad_idx)
        return imgs,targets

In [ ]:
!pip install scikit-learn

import sklearn
print(sklearn.__version__)  # Should print the installed version

 create train, test and validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Splitting the data
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1, random_state=42)

# Reset indices
train_df = train_df.reset_index(drop=True)  # Reset index for train_df
val_df = val_df.reset_index(drop=True)    # Reset index for val_df
test_df = test_df.reset_index(drop=True)   # Reset index for test_df

# Create datasets
train_dataset = CustomDataset(root_dir=image_data_location, captions_df=train_df, transform=transforms)
val_dataset = CustomDataset(root_dir=image_data_location, captions_df=val_df, transform=transforms)
test_dataset = CustomDataset(root_dir=image_data_location, captions_df=test_df, transform=transforms)

Creating Dataloaders with batches

In [ ]:
#writing the dataloader
#setting the constants
BATCH_SIZE = 35 #Was 4
NUM_WORKER = 2 #was 1

#token to represent the padding
pad_idx = dataset.vocab.stoi["<PAD>"]

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=True,
    collate_fn=CapsCollate(pad_idx=pad_idx,batch_first=True)
)

Creating Train Val and Test dataloaders

In [ ]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=True,
    collate_fn=CapsCollate(pad_idx=pad_idx, batch_first=True)
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=False,
    collate_fn=CapsCollate(pad_idx=pad_idx, batch_first=True)
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=False,
    collate_fn=CapsCollate(pad_idx=pad_idx, batch_first=True)
)

In [ ]:
#generating the iterator from the dataloader
dataiter = iter(data_loader)

#getting the next batch
batch = next(dataiter)

#unpacking the batch
images, captions = batch

#showing info of image in single batch
for i in range(BATCH_SIZE):
    img,cap = images[i],captions[i]
    #print(f"captions - {captions[i]}")
    caption_label = [dataset.vocab.itos[token] for token in cap.tolist()]
    eos_index = caption_label.index('<EOS>')
    caption_label = caption_label[1:eos_index]
    caption_label = ' '.join(caption_label)
    show_image(img,caption_label)
    plt.show()

Below code is to understand way of working of the  dataloader

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

Model Architecture of Encoder and Decoder with caption generation method

Need to modify this to CLIP model with self attention

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self,embed_size):
        super(EncoderCNN,self).__init__()
        resnet = models.resnet50(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad_(False)

        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.embed = nn.Linear(resnet.fc.in_features,embed_size)

    def forward(self,images):
        features = self.resnet(images)
#
        features = features.view(features.size(0),-1)
#
        features = self.embed(features)
#
        return features

class DecoderRNN(nn.Module):
    def __init__(self,embed_size,hidden_size,vocab_size,num_layers=1,drop_prob=0.3):
        super(DecoderRNN,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers=num_layers,batch_first=True)
        self.fcn = nn.Linear(hidden_size,vocab_size)
        self.drop = nn.Dropout(drop_prob)

    def forward(self,features, captions):
        # vectorize the caption
#
        embeds = self.embedding(captions[:,:-1])
#
        x = torch.cat((features.unsqueeze(1),embeds),dim=1)
#
        x,_ = self.lstm(x)
#
        x = self.fcn(x)
#
        return x

    def generate_caption(self,inputs,hidden=None,max_len=20,vocab=None): #hidden=None
        # Inference part
        # Given the image features generate the captions

        batch_size = inputs.size(0)

        captions = []

        if hidden is None:
          hidden = (torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(inputs.device),
                    torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(inputs.device))

         # Ensure inputs is 3D: [batch_size, 1, embed_size]
        inputs = inputs.unsqueeze(1)  # Add sequence length = 1

        for i in range(max_len):

            output, hidden = self.lstm(inputs, hidden)  # LSTM expects 3D input
            output = self.fcn(output.squeeze(1))  # Shape: [batch_size, vocab_size]
            #output = output.view(batch_size,-1)



            #select the word with most val
            predicted_word_idx = output.argmax(dim=1)

            #save the generated word
            captions.append(predicted_word_idx.item())

            #end if <EOS detected>
            if vocab.itos[predicted_word_idx.item()] == "<EOS>":
                break

            #send generated word as the next caption
            inputs = self.embedding(predicted_word_idx.unsqueeze(1)) # Shape: [batch_size, 1, embed_size]


        #covert the vocab idx to words and return sentence
        return [vocab.itos[idx] for idx in captions]


class EncoderDecoder(nn.Module):
    def __init__(self,embed_size,hidden_size,vocab_size,num_layers=1,drop_prob=0.3):
        super(EncoderDecoder,self).__init__()
        self.encoder = EncoderCNN(embed_size)
        self.decoder = DecoderRNN(embed_size,hidden_size,vocab_size,num_layers,drop_prob)

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Hyperparameters
embed_size = 512 #400
hidden_size = 512
vocab_size = len(dataset.vocab)
num_layers = 2
learning_rate = 0.0001
num_epochs = 2

In [ ]:
# initialize model, loss etc
model = EncoderDecoder(embed_size, hidden_size, vocab_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
import nltk
nltk.download('punkt')  # Download data for tokenization
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
# Ensure NLTK's tokenizer is downloaded (Run once)
nltk.download('punkt')
nltk.download('punkt_tab')

# Edited - SK
!pip install rouge-score jiwer
from rouge_score import rouge_scorer
from jiwer import wer

In [ ]:
special_tokens = ["<SOS>", "<UNK>", "<EOS>","<PAD>"]

In [ ]:
smoothie = SmoothingFunction().method4  # Choose a smoothing method
print("test1")

In [ ]:
print("Execution started")
from datetime import datetime
# Print current timestamp
print("Start Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

num_epochs = 5
train_losses = []
val_losses = []
val_blue_scores = []
train_vlue_scores = []
train_vlue_rouge_scores = []
train_vlue_wer_scores = []
val_rouge_scores = []
val_wer_scores = []
print_every = 6000
for epoch in range(1, num_epochs + 1):
    # Training Loop
    model.train()
    print("Epoch: {} StartTime: {}".format(epoch, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    train_loss = 0
    for idx, (image, captions) in enumerate(iter(train_loader)):
        image, captions = image.to(device), captions.to(device)
        optimizer.zero_grad()
        outputs = model(image, captions)
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        for img in image:
          reference_captions = token_to_sentence(train_dataset,captions[0])
          generated_captions = model.decoder.generate_caption(model.encoder(img.unsqueeze(0)),vocab=dataset.vocab)
          generated_captions = [token for token in generated_captions if token not in special_tokens]
          generated_captions_str = ' '.join(generated_captions)
          #Compute BLUE-4 Scores
          bleu4_score = sentence_bleu(reference_captions, generated_captions, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie )
          # Edited - SK
          scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
          rouge_scores = scorer.score(reference_captions, generated_captions_str)
          wer_score = wer(reference_captions, generated_captions_str)

          #bleu4_score = sentence_bleu(reference_captions, generated_captions, weights=(1, 0, 0, 0))
          train_vlue_scores.append(bleu4_score)
          train_vlue_rouge_scores.append(rouge_scores)
          train_vlue_wer_scores.append(wer_score)

    #calculate the train loss
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    print("Epoch: {} loss: {:.5f}".format(epoch, train_loss))

    # if (idx + 1) % print_every == 0:
    #   print("Epoch: {} loss: {:.5f}".format(epoch, loss.item()))

    # Validation Loop
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for val_image, val_captions in val_loader:
            val_image, val_captions = val_image.to(device), val_captions.to(device)
            outputs = model(val_image, val_captions)
            loss = criterion(outputs.view(-1, vocab_size), val_captions.view(-1))
            val_loss += loss.item()

            # Generate captions for the current batch of images
            for image in val_image:
              ref_caption = token_to_sentence(val_dataset,val_captions[0])
              print("Reference Caption:", ref_caption)
              generated_captions = model.decoder.generate_caption(model.encoder(image.unsqueeze(0)),vocab=dataset.vocab)
              generated_captions = [token for token in generated_captions if token not in special_tokens]
              generated_captions_str = ' '.join(generated_captions) # convert to string
              print("Generated Caption:", ' '.join(generated_captions))  # Print the generated caption
              # Compute BLEU-4 Score
              bleu4_score = sentence_bleu(ref_caption, generated_captions, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie)

              scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
              rouge_scores = scorer.score(ref_caption, generated_captions_str)
              wer_score = wer(ref_caption, generated_captions_str)
              #bleu4_score = sentence_bleu(ref_caption, generated_captions, weights=(1, 0, 0, 0))
              val_blue_scores.append(bleu4_score)
              val_rouge_scores.append(rouge_scores)
              val_wer_scores.append(wer_score)
              print("BLEU-4 Score: {:.5f}".format(bleu4_score))
              # print("ROUGE Score: {:.5f}".format(rouge_scores))
              for key, value in rouge_scores.items():
                print(f"ROUGE {key}: {value.fmeasure:.5f}")
              print("WER Score: {:.5f}".format(wer_score))


        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        print("Epoch: {} Validation loss: {:.5f}".format(epoch, val_loss))
        print("Epoch: {} EndTime: {}".format(epoch, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk

In [ ]:
print(f"Average train loss: {np.mean(train_losses)}")
print(f"Average validation loss: {np.mean(val_losses)}")
print(f"Average train BLEU-4 Score: {np.mean(train_vlue_scores)}")
print(f"Average val BLEU-4 Score: {np.mean(val_blue_scores)}")

# print(f"Average train Rouge Score: {np.mean(train_vlue_rouge_scores)}")
# print(f"Average val Rouge Score: {np.mean(val_rouge_scores)}")

import numpy as np

# Extract specific Rouge scores (e.g., 'rouge1' F1 scores)
rouge1_scores = [d['rouge1'].fmeasure for d in train_vlue_rouge_scores]

# Calculate the mean of the extracted scores
average_rouge1_score = np.mean(rouge1_scores)

# Print the average score
print(f"Average train Rouge-1 F1 Score: {average_rouge1_sco
print(f"Average train WER Score: {np.mean(train_vlue_wer_scores)}")
print(f"Average val WER Score: {np.mean(val_wer_scores)}")

In [ ]:
# Save the model
# torch.save(model.state_dict(), 'image_captioning_model.pth')
torch.save(model.state_dict(), 'image_captioning_model.pth')
torch.save(model.state_dict(), 'Flickr8K-resnet-lstm.pth')


In [ ]:
# prompt: provide code to download image_captioning_model.pth file from colab to local

from google.colab import files
files.download('image_captioning_model.pth')
files.download('Flickr8K-resnet-lstm.pth')


In [ ]:
import matplotlib.pyplot as plt
# Plot the training losses
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.show()


In [ ]:

# Plot the validation losses
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss Curve')
plt.legend()
plt.show()


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Ensure NLTK's tokenizer is downloaded (Run once)
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
# Set model to evaluation mode
blu_scores = []

model.eval()

# Disable gradient calculation
with torch.no_grad():
    for test_image, test_captions in test_loader:
        test_image, test_captions = test_image.to(device), test_captions.to(device)

        for i, image in enumerate(test_image):
            # Generate captions using the model
            generated_caption = model.decoder.generate_caption(
                model.encoder(image.unsqueeze(0)), vocab=dataset.vocab
            )

            # Convert generated caption to tokens
            generated_caption = [token for token in generated_caption if token not in special_tokens]
            generated_caption = " ".join(generated_caption)
            reference_captions = token_to_sentence(test_dataset,test_captions[0])


            # Compute BLEU-4 Score
            bleu4_score = sentence_bleu(reference_captions, generated_caption, weights=(0.25, 0.25, 0.25, 0.25))
            blu_scores.append(bleu4_score)

            # Print results
            print(f"Reference Captions: {reference_captions}")
            print(f"Generated Caption: {generated_caption}")
            print(f"BLEU-4 Score: {bleu4_score:.4f}\n")

print(f"Average BLEU-4 Score: {np.mean(blu_scores):.4f}")

In [ ]:


# Assuming 'blu_scores' is a list of BLEU scores from the previous code
# and you have a corresponding list of image indices or identifiers.

# Example image identifiers (replace with your actual identifiers)
image_ids = list(range(len(blu_scores)))

plt.figure(figsize=(10, 10))  # Adjust figure size as needed
plt.scatter(image_ids, blu_scores, color='blue', label='BLEU Scores')
plt.xlabel('Image ID')
plt.ylabel('BLEU Score')
plt.title('BLEU Scores for Images')
plt.legend()
plt.grid(True)
plt.show()
